In [ ]:
!pip show fastai
!pip show torch


Name: fastai
Version: 2.7.19
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: fastcore, fastdownload, fastprogress, matplotlib, packaging, pandas, pillow, pip, pyyaml, requests, scikit-learn, scipy, spacy, torch, torchvision
Required-by: 
Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-c

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from fastai.vision.all import *
from PIL import Image as PIL_Original

# -- Step 1: Patch PILImage.create for dummy images --
# def fake_open_image(fn):
#     dummy_tensor = torch.zeros(3, 224, 224).byte()
#     dummy_tensor = dummy_tensor.permute(1, 2, 0)
#     import PIL.Image
#     pil_img = PIL.Image.fromarray(dummy_tensor.numpy())
#     return PILImage(pil_img)

#PILImage.create = fake_open_image  # Use fake image loader

# -- Step 2: Dummy DataFrame --
dummy_data = {
    'image': ['714'] * 5,
    'level': [0, 1, 2, 3, 4]
}
dummy_df = pd.DataFrame(dummy_data)

# -- Step 3: Create dls --
dls = ImageDataLoaders.from_df(
    dummy_df,
    path='/content',
    fn_col='image',
    label_col='level',
    suff='.jpg',
    valid_pct=0.0,
    bs=1,
    item_tfms=[],
    batch_tfms=[],
    shuffle=False
)

In [3]:
dls.c

5

In [4]:
dls.vocab

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

In [5]:
from fastai.vision.all import *

import torch
import torchvision.models as models

learn = vision_learner(dls, resnet34, path='.',
    loss_func=FocalLoss(),
    metrics=[accuracy],  n_out=5)

# Load weights with map_location in torch.load
checkpoint = torch.load('/content/drive/MyDrive/test.pth', weights_only=False, map_location=torch.device('cpu'))

# checkpoint is a dict with keys: 'model', 'opt', etc.
state_dict = checkpoint['model']  # extract just the model weights

learn.model.load_state_dict(state_dict)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 84.5MB/s]


<All keys matched successfully>

In [6]:
print(learn.model)


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [7]:
learn.dls.vocab = [0,1,2,3,4]  # or whatever your classes are


In [8]:
img_path = '/content/714.jpg'  # your image path

pred_class, pred_idx, probs = learn.predict(img_path)
probs = F.softmax(probs, dim=0)
print(probs)

probs_softmax = F.softmax(probs, dim=0)
print(probs_softmax)
print(f"Sum of probs: {probs_softmax.sum().item()}")  # Should be 1.0
max_prob, max_idx = probs_softmax.max(0)
print(f"Prediction idx: {max_idx.item()}, Probability: {max_prob.item():.4f}")

tensor([0.1487, 0.1741, 0.4952, 0.0571, 0.1249])
tensor([0.1877, 0.1925, 0.2654, 0.1712, 0.1832])
Sum of probs: 1.0000001192092896
Prediction idx: 2, Probability: 0.2654


In [9]:
img_path = '/content/10.jpg'  # your image path

pred_class, pred_idx, probs = learn.predict(img_path)
probs = F.softmax(probs, dim=0)
print(probs)

probs_softmax = F.softmax(probs, dim=0)
print(probs_softmax)
print(f"Sum of probs: {probs_softmax.sum().item()}")  # Should be 1.0
max_prob, max_idx = probs_softmax.max(0)
print(f"Prediction idx: {max_idx.item()}, Probability: {max_prob.item():.4f}")

tensor([0.1970, 0.1879, 0.4221, 0.0546, 0.1385])
tensor([0.1979, 0.1961, 0.2478, 0.1716, 0.1866])
Sum of probs: 1.0
Prediction idx: 2, Probability: 0.2478


In [15]:
import torch

# Set the model to evaluation mode
learn.model.eval()

# Example dummy input (adjust size to your model input, e.g., 3x224x224)
dummy_input = torch.randn(1, 3, 224, 224)

# Export to ONNX
torch.onnx.export(learn.model, dummy_input, "model.onnx",
                  export_params=True, opset_version=11,
                  do_constant_folding=True,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})


# Installing Requirements for ONNX-TF2

In [12]:
!pip install onnx onnx-tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 42.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [13]:
!pip install onnx2tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.0/448.0 kB 28.7 MB/s eta 0:00:00


In [14]:
!pip install onnx-graphsurgeon --extra-index-url https://pypi.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.9 MB/s eta 0:00:00


In [16]:
!pip install ai-edge-litert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 25.8 MB/s eta 0:00:00


In [17]:
!python3 -m pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com
!pip install sng4onnx

Looking in indexes: https://pypi.ngc.nvidia.com


# Converting ONNX to Tensorflow

In [18]:
!onnx2tf --input_onnx_file_path model.onnx --output_folder_path model_tf

E0000 00:00:1749456471.976536    1515 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749456472.048139    1515 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Model optimizing started ============================================================
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/onnx2tf.py", line 668, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

In [ ]:
# !pip install tensorflow==2.15.0 keras==2.15.0 tensorflow-addons==0.23.0

# Tensorflow to TF-Lite

In [19]:
import tensorflow as tf

# Path to your saved TensorFlow model directory
saved_model_dir = "model_tf"

# Create the converter
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# (Optional) Enable optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the converted model to a file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved as model.tflite")

TensorFlow Lite model saved as model.tflite


# Tensorflow Testing

In [20]:
import tensorflow as tf

# Load the SavedModel
model = tf.saved_model.load("model_tf")

# Get the inference function
infer = model.signatures["serving_default"]

In [21]:
# Load image
img_path = "/content/714.jpg"  # Replace with your image path
img = Image.open(img_path).convert("RGB")
img = img.resize((224, 224))  # Resize to model's input size

# Convert to numpy and normalize (if needed)
img_array = np.array(img).astype(np.float32) / 255.0  # normalize if model expects it
input_data = np.expand_dims(img_array, axis=0)  # Shape: (1, 224, 224, 3)

In [22]:
# Use input key from step above, e.g., "input_0"
output = infer(tf.convert_to_tensor(input_data))  # or {'input_0': tf.convert_to_tensor(input_data)}

# Extract and print results
for key, value in output.items():
    print(f"{key}: {value.numpy()}")


output_0: [[ 2.674038   0.7281837  0.8120449 -3.8373568 -4.3084555]]


In [23]:
import numpy as np

logits = list(output.values())[0].numpy()
exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

print("Probabilities:", probs)


Probabilities: [[7.68850923e-01 1.09841995e-01 1.19450733e-01 1.14282430e-03
  7.13483256e-04]]


In [24]:
predicted_class = np.argmax(probs, axis=1)
print(f"Predicted class: {predicted_class}")

Predicted class: [0]


In [25]:
!zip -r model_tf_zip.zip model_tf

  adding: model_tf/ (stored 0%)
  adding: model_tf/variables/ (stored 0%)
  adding: model_tf/variables/variables.data-00000-of-00001 (deflated 84%)
  adding: model_tf/variables/variables.index (deflated 33%)
  adding: model_tf/fingerprint.pb (stored 0%)
  adding: model_tf/saved_model.pb (deflated 7%)
  adding: model_tf/assets/ (stored 0%)
  adding: model_tf/model_float16.tflite (deflated 8%)
  adding: model_tf/model_float32.tflite (deflated 7%)


# Tensorflow Lite Test

In [26]:
import numpy as np
import tensorflow as tf
from PIL import Image
import urllib.request

def load_image(image_path, size=(224, 224)):
    img = Image.open(image_path).convert("RGB")
    img = img.resize(size)
    img_array = np.array(img).astype(np.float32) / 255.0  # normalize if needed
    return np.expand_dims(img_array, axis=0)  # shape: (1, H, W, C)

def run_tflite_model(tflite_model_path, input_data):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

# Example usage in Colab:

# 1. Upload your TFLite model file, or mount Google Drive and provide path
# from google.colab import files
# uploaded = files.upload()
# tflite_model_path = list(uploaded.keys())[0]

# Or if your model is in Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# tflite_model_path = "/content/drive/MyDrive/model.tflite"

# For demo: download a sample image
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Golden_Retriever_medium-to-light-coat.jpg/320px-Golden_Retriever_medium-to-light-coat.jpg"
image_path = "/content/714.jpg"
# urllib.request.urlretrieve(img_url, image_path)

# Provide your tflite model path here
tflite_model_path = "model.tflite"  # replace with your actual model file path

# Load image and preprocess
input_data = load_image(image_path)

# Run inference
output = run_tflite_model(tflite_model_path, input_data)

# Softmax to get probabilities
probs = tf.nn.softmax(output).numpy()

print("Output probabilities:", probs)
predicted_class = np.argmax(probs, axis=1)
print("Predicted class index:", predicted_class)


Output probabilities: [[7.7211374e-01 1.1819353e-01 1.0823437e-01 9.3848584e-04 5.1996263e-04]]
Predicted class index: [0]
